## Import requried libs

In [108]:
from cassandra.cluster import Cluster
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, ShuffleSplit
from sklearn.metrics import *
%matplotlib inline

In [109]:
from sklearn.feature_selection import RFE, RFECV
from sklearn.utils import resample

In [110]:
import pyspark

In [111]:
from pyspark.sql import SparkSession

In [112]:
from pyspark.sql.functions import col,isnan,when,count

In [113]:
from pyspark.sql.functions import *

In [114]:
from pyspark.sql.types import *
from pyspark.sql.functions import col

In [115]:
spark = SparkSession \
    .builder \
    .appName("Credit card fraud detection") \
    .getOrCreate()

In [116]:
spark

In [117]:
df = spark.read.option('header', 'true').csv('credit_card_transaction_data.csv')

In [118]:
df.show()

+--------------+----+----+----+-----+---+-----+-------+------------------+--------------------+-------------+--------------+-------+----+------+-------+
|Transaction_ID|User|Card|Year|Month|Day| Time| Amount|          Use_Chip|       Merchant_Name|Merchant_City|Merchant_State|    Zip| MCC|Errors|IsFraud|
+--------------+----+----+----+-----+---+-----+-------+------------------+--------------------+-------------+--------------+-------+----+------+-------+
|             0|   0|   0|2002|    9|  1|06:21|$134.09| Swipe Transaction| 3527213246127876953|     La Verne|            CA|91750.0|5300|  null|     No|
|             1|   0|   0|2002|    9|  1|06:42| $38.48| Swipe Transaction| -727612092139916043|Monterey Park|            CA|91754.0|5411|  null|     No|
|             2|   0|   0|2002|    9|  2|06:22|$120.34| Swipe Transaction| -727612092139916043|Monterey Park|            CA|91754.0|5411|  null|     No|
|             3|   0|   0|2002|    9|  2|17:45|$128.95| Swipe Transaction| 3414527

In [119]:
df.count()

24386900

In [120]:
df.head(3)

[Row(Transaction_ID='0', User='0', Card='0', Year='2002', Month='9', Day='1', Time='06:21', Amount='$134.09', Use_Chip='Swipe Transaction', Merchant_Name='3527213246127876953', Merchant_City='La Verne', Merchant_State='CA', Zip='91750.0', MCC='5300', Errors=None, IsFraud='No'),
 Row(Transaction_ID='1', User='0', Card='0', Year='2002', Month='9', Day='1', Time='06:42', Amount='$38.48', Use_Chip='Swipe Transaction', Merchant_Name='-727612092139916043', Merchant_City='Monterey Park', Merchant_State='CA', Zip='91754.0', MCC='5411', Errors=None, IsFraud='No'),
 Row(Transaction_ID='2', User='0', Card='0', Year='2002', Month='9', Day='2', Time='06:22', Amount='$120.34', Use_Chip='Swipe Transaction', Merchant_Name='-727612092139916043', Merchant_City='Monterey Park', Merchant_State='CA', Zip='91754.0', MCC='5411', Errors=None, IsFraud='No')]

In [122]:
type(df)

pyspark.sql.dataframe.DataFrame

In [123]:
df.printSchema()

root
 |-- Transaction_ID: string (nullable = true)
 |-- User: string (nullable = true)
 |-- Card: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Day: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- Use_Chip: string (nullable = true)
 |-- Merchant_Name: string (nullable = true)
 |-- Merchant_City: string (nullable = true)
 |-- Merchant_State: string (nullable = true)
 |-- Zip: string (nullable = true)
 |-- MCC: string (nullable = true)
 |-- Errors: string (nullable = true)
 |-- IsFraud: string (nullable = true)



### get statistics about null/nan values 

In [124]:
df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df.columns]).show()

DataFrame[Transaction_ID: bigint, User: bigint, Card: bigint, Year: bigint, Month: bigint, Day: bigint, Time: bigint, Amount: bigint, Use_Chip: bigint, Merchant_Name: bigint, Merchant_City: bigint, Merchant_State: bigint, Zip: bigint, MCC: bigint, Errors: bigint, IsFraud: bigint]

### get data types of each attributes

In [125]:
df.dtypes

[('Transaction_ID', 'string'),
 ('User', 'string'),
 ('Card', 'string'),
 ('Year', 'string'),
 ('Month', 'string'),
 ('Day', 'string'),
 ('Time', 'string'),
 ('Amount', 'string'),
 ('Use_Chip', 'string'),
 ('Merchant_Name', 'string'),
 ('Merchant_City', 'string'),
 ('Merchant_State', 'string'),
 ('Zip', 'string'),
 ('MCC', 'string'),
 ('Errors', 'string'),
 ('IsFraud', 'string')]

### get statitical description of the data

In [127]:

df['Year', 'Month', 'Day'].describe().show()

+-------+------------------+-----------------+------------------+
|summary|              Year|            Month|               Day|
+-------+------------------+-----------------+------------------+
|  count|          24386900|         24386900|          24386900|
|   mean|2011.9551699067943| 6.52506357921671|15.718122721625134|
| stddev| 5.105920688923643|3.472354834572916| 8.794073288462577|
|    min|              1991|                1|                 1|
|    max|              2020|                9|                 9|
+-------+------------------+-----------------+------------------+



In [128]:
df['Amount', 'Use_Chip', 'Merchant_City', 'Merchant_State'].describe().show()

+-------+--------+-----------------+-------------+--------------+
|summary|  Amount|         Use_Chip|Merchant_City|Merchant_State|
+-------+--------+-----------------+-------------+--------------+
|  count|24386900|         24386900|     24386900|      21666079|
|   mean|    null|             null|         null|          null|
| stddev|    null|             null|         null|          null|
|    min|  $-0.00| Chip Transaction|       ONLINE|            AA|
|    max| $999.97|Swipe Transaction|       Zwolle|      Zimbabwe|
+-------+--------+-----------------+-------------+--------------+



In [129]:
df['MCC', 'Errors', 'IsFraud', 'Zip'].describe().show()

+-------+-----------------+-----------------+--------+------------------+
|summary|              MCC|           Errors| IsFraud|               Zip|
+-------+-----------------+-----------------+--------+------------------+
|  count|         24386900|           388431|24386900|          21508765|
|   mean|5561.171253336833|             null|    null| 50956.44211506332|
| stddev|879.3154327182834|             null|    null|29397.065949063595|
|    min|             1711|         Bad CVV,|      No|           10001.0|
|    max|             9402|Technical Glitch,|     Yes|           99928.0|
+-------+-----------------+-----------------+--------+------------------+



Analysis : 
Year : the transaction data has the range of year 1991 - 2020 : datatype of the year : int

Month : the transaction data has the range of month Jan - Sept : datatype of the month : int

Day: the transaction data has the range of day 1 - 9 : datatype of the day : int

Amount : Amount attributes contians the $ sign which makes it string type attribute. This attribute should be refined to have float type. 

Merchant_State : This attribute contians lesser count so it contians nan or empty string. 

MCC : Mechant Category Codes - This code is provided by the credit card issuer to categorize the transaction made by the customer. 

Errors : The lesser count of this attribut suggests that it contians null values. 

Zip : the datatype of this attribute is float which needs to be convered to int. 

IsFraud : this is the target categorical variable 


Numerical attributes : Year, Month, day, Amount, MCC, Zip 

Categorical attributes: Merchant_State, Merchant_City, Errors, IsFraud, Use_Chip

### Convert amount to numeric

##### Remove $ sign 

In [130]:
df = df.withColumn('Amount', translate('Amount', '$', ''))

In [131]:
df.select(col("Amount")).show()

+------+
|Amount|
+------+
|134.09|
| 38.48|
|120.34|
|128.95|
|104.71|
| 86.19|
| 93.84|
|123.50|
| 61.72|
| 57.10|
| 76.07|
| 53.91|
|110.37|
|117.05|
| 45.30|
| 29.34|
|147.45|
| 27.75|
| 76.57|
| 22.56|
+------+
only showing top 20 rows



In [132]:
df = df.withColumn("Amount",round(df.Amount.cast(DoubleType()),2))

In [133]:
df.dtypes

[('Transaction_ID', 'string'),
 ('User', 'string'),
 ('Card', 'string'),
 ('Year', 'string'),
 ('Month', 'string'),
 ('Day', 'string'),
 ('Time', 'string'),
 ('Amount', 'double'),
 ('Use_Chip', 'string'),
 ('Merchant_Name', 'string'),
 ('Merchant_City', 'string'),
 ('Merchant_State', 'string'),
 ('Zip', 'string'),
 ('MCC', 'string'),
 ('Errors', 'string'),
 ('IsFraud', 'string')]

### Convert Year, Month, Day, Zip, MCC in to Integer type

In [134]:
df = df.withColumn("Year", df["Year"].cast(IntegerType()))

In [135]:
df = df.withColumn("Month", df["Month"].cast(IntegerType()))
df = df.withColumn("Day", df["Day"].cast(IntegerType()))
df = df.withColumn("Zip", df["Zip"].cast(IntegerType()))
df = df.withColumn("MCC", df["MCC"].cast(IntegerType()))

In [136]:
df = df.withColumn("Transaction_ID", df["Transaction_ID"].cast(IntegerType()))
df = df.withColumn("User", df["User"].cast(IntegerType()))
df = df.withColumn("Card", df["Card"].cast(IntegerType()))

In [137]:
df.dtypes

[('Transaction_ID', 'int'),
 ('User', 'int'),
 ('Card', 'int'),
 ('Year', 'int'),
 ('Month', 'int'),
 ('Day', 'int'),
 ('Time', 'string'),
 ('Amount', 'double'),
 ('Use_Chip', 'string'),
 ('Merchant_Name', 'string'),
 ('Merchant_City', 'string'),
 ('Merchant_State', 'string'),
 ('Zip', 'int'),
 ('MCC', 'int'),
 ('Errors', 'string'),
 ('IsFraud', 'string')]

### Convert time (hr : minute) formate to only hour

In [138]:
df = df.withColumn('hour', split(df['Time'], ':').getItem(0))

In [139]:
df = df.withColumn("hour", df["hour"].cast(IntegerType()))

In [140]:
df.select(col("hour")).show()

+----+
|hour|
+----+
|   6|
|   6|
|   6|
|  17|
|   6|
|  13|
|   5|
|   6|
|   6|
|   9|
|  20|
|  20|
|   6|
|   6|
|   6|
|   9|
|   6|
|   6|
|  13|
|  22|
+----+
only showing top 20 rows



### Drop time attributes

In [141]:
df = df.drop('Time')

### Analyze merchant state and zip code

#### Numbers of null values in merchant_state attribute

In [50]:
'''
data_state_na = df.select([when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            col(c).isNull() | \
                            isnan(c), c 
                           ).alias(c)
                    for c in ['Merchant_State']])
'''

In [142]:
data_state_na = df.filter(col("Merchant_State").isNull())

In [143]:
data_state_na.count()

2720821

2720821 out of 24386900 contains null values in the merchant_state attributes. 

#### How many of them are the fraud transactions ?

In [144]:
data_state_na.show()

+--------------+----+----+----+-----+---+------+------------------+--------------------+-------------+--------------+----+----+--------------------+-------+----+
|Transaction_ID|User|Card|Year|Month|Day|Amount|          Use_Chip|       Merchant_Name|Merchant_City|Merchant_State| Zip| MCC|              Errors|IsFraud|hour|
+--------------+----+----+----+-----+---+------+------------------+--------------------+-------------+--------------+----+----+--------------------+-------+----+
|            11|   0|   0|2002|    9|  5| 53.91|Online Transaction|-9092677072201095172|       ONLINE|          null|null|4900|                null|     No|  20|
|            24|   0|   0|2002|    9|  9| 144.9|Online Transaction|-8338381919281017248|       ONLINE|          null|null|4899|                null|     No|  20|
|            85|   0|   0|2002|    9| 30|127.32|Online Transaction|-7421093378627544099|       ONLINE|          null|null|5311|                null|     No|   6|
|            99|   0|   0|20

In [145]:
data_state_na.filter(col("IsFraud") == 'Yes').count()

18349

In [146]:
data_state_na.filter(col('Merchant_City') == ' ONLINE').count()

2720821

In [147]:
df.filter(col('Zip').isNull() & col('Merchant_State').isNotNull()).count()

157314

getting nan for the merchant state whenever the transaction made online and no data found for their state and zip code is also nan at that time. so replacing merchant state with online is the good choice. 


Zip column contains 11% of missing values, and that can be imputed if we find zip value for same merchant state and city for which we already have zip values. 

In [148]:
data_zip_notna = df.filter(col('Zip').isNotNull() & col('Merchant_State').isNotNull())

In [149]:
data_zip_na_state_notna = df.filter(col('Zip').isNull() & col('Merchant_State').isNotNull())

In [150]:
data_zip_na_state_notna.select('Merchant_State').distinct().collect()

[Row(Merchant_State='Russia'),
 Row(Merchant_State='Sweden'),
 Row(Merchant_State='Philippines'),
 Row(Merchant_State='Singapore'),
 Row(Merchant_State='Malaysia'),
 Row(Merchant_State='Turkey'),
 Row(Merchant_State='Germany'),
 Row(Merchant_State='Cambodia'),
 Row(Merchant_State='France'),
 Row(Merchant_State='Greece'),
 Row(Merchant_State='Taiwan'),
 Row(Merchant_State='Algeria'),
 Row(Merchant_State='Slovakia'),
 Row(Merchant_State='Argentina'),
 Row(Merchant_State='Belgium'),
 Row(Merchant_State='Finland'),
 Row(Merchant_State='Ghana'),
 Row(Merchant_State='Sierra Leone'),
 Row(Merchant_State='Peru'),
 Row(Merchant_State='Benin'),
 Row(Merchant_State='China'),
 Row(Merchant_State='India'),
 Row(Merchant_State='Tuvalu'),
 Row(Merchant_State='Croatia'),
 Row(Merchant_State='Nigeria'),
 Row(Merchant_State='Italy'),
 Row(Merchant_State='Lithuania'),
 Row(Merchant_State='Norway'),
 Row(Merchant_State='Spain'),
 Row(Merchant_State='Denmark'),
 Row(Merchant_State='Bangladesh'),
 Row(Merch

In [151]:
state_list = data_zip_notna.select('Merchant_State').distinct().collect()

In [152]:
state_list

[Row(Merchant_State='AZ'),
 Row(Merchant_State='SC'),
 Row(Merchant_State='LA'),
 Row(Merchant_State='MN'),
 Row(Merchant_State='NJ'),
 Row(Merchant_State='DC'),
 Row(Merchant_State='OR'),
 Row(Merchant_State='VA'),
 Row(Merchant_State='RI'),
 Row(Merchant_State='KY'),
 Row(Merchant_State='WY'),
 Row(Merchant_State='NH'),
 Row(Merchant_State='MI'),
 Row(Merchant_State='NV'),
 Row(Merchant_State='WI'),
 Row(Merchant_State='ID'),
 Row(Merchant_State='CA'),
 Row(Merchant_State='NE'),
 Row(Merchant_State='CT'),
 Row(Merchant_State='MT'),
 Row(Merchant_State='NC'),
 Row(Merchant_State='VT'),
 Row(Merchant_State='MD'),
 Row(Merchant_State='DE'),
 Row(Merchant_State='MO'),
 Row(Merchant_State='IL'),
 Row(Merchant_State='ME'),
 Row(Merchant_State='WA'),
 Row(Merchant_State='ND'),
 Row(Merchant_State='MS'),
 Row(Merchant_State='AL'),
 Row(Merchant_State='IN'),
 Row(Merchant_State='OH'),
 Row(Merchant_State='TN'),
 Row(Merchant_State='IA'),
 Row(Merchant_State='NM'),
 Row(Merchant_State='PA'),
 

for which we don't find zip code even though we have merchant state : the reason is merchant state doesn't contain "State" it contains "Country". so create new column Merchant Country and copy Merchant state value in Merchant country for which zip code is not avialable and merchant state is available. and impute "USA" as a country for which Zip code is avilable and then drop zip code column.

In [153]:
df = df.withColumn('Zip', when(col('Zip').isNull(), 0).otherwise(col('Zip')))

In [154]:
df.filter(isnan('Zip')).count()

0

In [155]:
df = df.withColumn('Merchant_State', trim(df.Merchant_State))

In [156]:
df = df.withColumn('Merchant_City', trim(df.Merchant_City))

In [157]:
df = df.withColumn("Merchant_Country",when((col("Zip") == 0) & (col('Merchant_State') != ""), col('Merchant_State')).
              when((col("Zip") != 0) & (col('Merchant_State') != ""), 'USA').
              otherwise('ONLINE'))

In [158]:
df = df.withColumn('Merchant_State', when(col('Merchant_State').isNull(), 'ONLINE'))

### Analyze Error column

In [159]:
error_list = df.select('Errors').distinct().collect()

In [160]:
error_list

[Row(Errors='Bad Expiration,Bad CVV,'),
 Row(Errors='Insufficient Balance,Technical Glitch,'),
 Row(Errors='Bad CVV,Technical Glitch,'),
 Row(Errors='Bad CVV,'),
 Row(Errors=None),
 Row(Errors='Technical Glitch,'),
 Row(Errors='Bad Card Number,Bad CVV,'),
 Row(Errors='Bad CVV,Insufficient Balance,'),
 Row(Errors='Bad Expiration,Insufficient Balance,'),
 Row(Errors='Bad Expiration,'),
 Row(Errors='Bad Card Number,Bad Expiration,Technical Glitch,'),
 Row(Errors='Bad Card Number,Bad Expiration,'),
 Row(Errors='Bad PIN,Insufficient Balance,'),
 Row(Errors='Bad Expiration,Technical Glitch,'),
 Row(Errors='Bad PIN,Technical Glitch,'),
 Row(Errors='Bad PIN,'),
 Row(Errors='Insufficient Balance,'),
 Row(Errors='Bad Card Number,Insufficient Balance,'),
 Row(Errors='Bad Card Number,'),
 Row(Errors='Bad Zipcode,'),
 Row(Errors='Bad Card Number,Technical Glitch,'),
 Row(Errors='Bad Zipcode,Insufficient Balance,'),
 Row(Errors='Bad Zipcode,Technical Glitch,'),
 Row(Errors='Bad Card Number,Bad Expir

In [161]:
df.filter(isnan('Errors') | col('Errors').isNull()).count()

23998469

Error column contains many missing values and most of them are not directly relavant to the transaction being fraud or not as many error value being null is related to the transaction being legitimate. 

### Impute missing value of merchant state and merchant country 

In [162]:
df = df.withColumn('Merchant_State', when(col('Merchant_State').isNull(), 'ONLINE'))
df = df.withColumn('Merchant_Country', when(col('Merchant_Country').isNull(), 'ONLINE'))

In [163]:
df = df.withColumn('Merchant_State', when(col('Merchant_State') == 'null', 'ONLINE'))
df = df.withColumn('Merchant_Country', when(isnan('Merchant_Country'), 'ONLINE'))

In [172]:
df = df.na.fill('ONLINE',['Merchant_State','Merchant_Country'])

In [173]:
df.dtypes

[('Year', 'int'),
 ('Month', 'int'),
 ('Day', 'int'),
 ('Amount', 'double'),
 ('Use_Chip', 'string'),
 ('Merchant_City', 'string'),
 ('Merchant_State', 'string'),
 ('MCC', 'int'),
 ('IsFraud', 'string'),
 ('hour', 'int'),
 ('Merchant_Country', 'string')]

Drop Error and zip column 
user, Card and Merchant Name will not be useful in training ML models. 

In [166]:
df = df.drop('Errors', 'Zip', 'Transaction_ID', 'User', 'Card', 'Merchant_Name')

### Convert categorical data to numeric data 

In [167]:
from pyspark.ml.feature import StringIndexer

In [174]:
df.show()

+----+-----+---+------+------------------+-------------+--------------+----+-------+----+----------------+
|Year|Month|Day|Amount|          Use_Chip|Merchant_City|Merchant_State| MCC|IsFraud|hour|Merchant_Country|
+----+-----+---+------+------------------+-------------+--------------+----+-------+----+----------------+
|2002|    9|  1|134.09| Swipe Transaction|     La Verne|        ONLINE|5300|     No|   6|          ONLINE|
|2002|    9|  1| 38.48| Swipe Transaction|Monterey Park|        ONLINE|5411|     No|   6|          ONLINE|
|2002|    9|  2|120.34| Swipe Transaction|Monterey Park|        ONLINE|5411|     No|   6|          ONLINE|
|2002|    9|  2|128.95| Swipe Transaction|Monterey Park|        ONLINE|5651|     No|  17|          ONLINE|
|2002|    9|  3|104.71| Swipe Transaction|     La Verne|        ONLINE|5912|     No|   6|          ONLINE|
|2002|    9|  3| 86.19| Swipe Transaction|Monterey Park|        ONLINE|5970|     No|  13|          ONLINE|
|2002|    9|  4| 93.84| Swipe Transac

In [181]:
indexer = StringIndexer(inputCols=["Use_Chip", "Merchant_City","Merchant_State","Merchant_Country", "IsFraud"], outputCols=
                        ["Use_chip", "Merchant_city","Merchant_state","Merchant_country", "isFraud"])
df_transform = indexer.fit(df).transform(df)
df_transform.select("Use_chip", "Merchant_city","Merchant_state","Merchant_country", "isFraud").show()

+--------+-------------+--------------+----------------+-------+
|Use_chip|Merchant_city|Merchant_state|Merchant_country|isFraud|
+--------+-------------+--------------+----------------+-------+
|     0.0|        517.0|           0.0|             0.0|    0.0|
|     0.0|        746.0|           0.0|             0.0|    0.0|
|     0.0|        746.0|           0.0|             0.0|    0.0|
|     0.0|        746.0|           0.0|             0.0|    0.0|
|     0.0|        517.0|           0.0|             0.0|    0.0|
|     0.0|        746.0|           0.0|             0.0|    0.0|
|     0.0|        746.0|           0.0|             0.0|    0.0|
|     0.0|        746.0|           0.0|             0.0|    0.0|
|     0.0|        746.0|           0.0|             0.0|    0.0|
|     0.0|        517.0|           0.0|             0.0|    0.0|
|     0.0|        517.0|           0.0|             0.0|    0.0|
|     2.0|          0.0|           0.0|             0.0|    0.0|
|     0.0|        165.0| 

In [183]:
df_transform.select('isFraud').distinct().collect()

[Row(isFraud=0.0), Row(isFraud=1.0)]

In [184]:
df_transform = df_transform.withColumn("isFraud", df_transform["isFraud"].cast(IntegerType()))
df_transform = df_transform.withColumn("Use_chip", df_transform["Use_chip"].cast(IntegerType()))
df_transform = df_transform.withColumn("Merchant_city", df_transform["Merchant_city"].cast(IntegerType()))
df_transform = df_transform.withColumn("Merchant_state", df_transform["Merchant_state"].cast(IntegerType()))
df_transform = df_transform.withColumn("Merchant_country", df_transform["Merchant_country"].cast(IntegerType()))

In [185]:
df_transform.dtypes

[('Year', 'int'),
 ('Month', 'int'),
 ('Day', 'int'),
 ('Amount', 'double'),
 ('Use_chip', 'int'),
 ('Merchant_city', 'int'),
 ('Merchant_state', 'int'),
 ('MCC', 'int'),
 ('isFraud', 'int'),
 ('hour', 'int'),
 ('Merchant_country', 'int')]

### Split the data in to train, validation and test data and save in the directory 

In [189]:
train_data,test_data=df_transform.randomSplit([0.75,0.25], 24)

In [195]:
train_data.select('isFraud').distinct().collect()

[Row(isFraud=1), Row(isFraud=0)]

In [198]:
train_data.groupBy('isFraud').count().show()

+-------+--------+
|isFraud|   count|
+-------+--------+
|      1|   22289|
|      0|18271549|
+-------+--------+



In [190]:
feature = train_data.columns

In [192]:
feature.remove('isFraud')